# Wildfire Data Extraction - Del Rio, Texas

This code extracts the fires data from the wildfire dataset and filters it based on certain conditions using the city assigned. The purpose of this code  to  find the fires which  are within 1250 miles of the assigned city.
My Assigned City: Del Rio, Texas

The data extraction code extracts data from the USGS Wildfire data and I found that there are 70861 fires which are  under 1250 miles from Del Rio based on distance.

I have used the sample code from the notebook provided by Prof as a reference to carry out the below tasks.

## Import Libraries

In [186]:
import pandas as  pd
import plotly.express as px
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of California wildfires extracted from the main wildfire dataset.
#
import geojson
import concurrent.futures

## Data Extraction

Defining file path which is downloaded from https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81

File: USGS_Wildland_Fire_Combined_Dataset.json

Also defining the Latitude and Longitude for my assigned city: Del Rio Texas

In [190]:
# Data path
DATA_FILE = os.path.join("/Users/stlp/Documents/DATA-512/GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json")
print(f"{DATA_FILE=}")

# Defining Lat Lon for my city
Del_Rio_Lat_Lon = (29.3692, -100.8908)

DATA_FILE='/Users/stlp/Documents/DATA-512/GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json'


Geojson to read the data and counting the number of features/fires in the Data

In [3]:
geojson_file = open(DATA_FILE,"r")
gj_data = geojson.load(geojson_file)
geojson_file.close()
#
#    Print the keys and number of features from the object
#
gj_keys = list(gj_data.keys())
print("The loaded JSON dictionary has the following keys:")
print(gj_keys)

print("\nFound "+str(len(gj_data['features']))+" features in the variable gj_data")

The loaded JSON dictionary has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']

Found 135061 features in the variable gj_data


Defining functions to calculate the rings and distance of fires from the desired US city  

In [4]:
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point

Extracting the fires specific to 3 filters:
1. The estimate only considers the last 60 years of wildland fires (1963-2023).
2. The estimate only considers fires that are within 1250 miles of your assigned city.
3. An annual fire season will run from May 1st through October 31st.

The features are appended into the list based on these filters for my city: Del Rio

In [ ]:
del_rio_features = list()
excluded_list = list()

def calculate_closest_distance(wf_feature):
    wf_year = wf_feature['attributes']['Fire_Year']
    wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
    wf_size = wf_feature['attributes']['GIS_Acres']
    wf_type = wf_feature['attributes']['Assigned_Fire_Type']
    ring_data = wf_feature['geometry']['rings'][0]

    if wf_year >= 1963:
        distance = shortest_distance_from_place_to_fire_perimeter(Del_Rio_Lat_Lon, ring_data)
        # print(f"The closest distance of fire '{wf_name}' ({wf_size:1.2f} acres) from {wf_year} was {distance[0]:1.2f} miles to Del Rio")

        if distance[0] <= 1250:
            wf_feature['attributes']['fire_lat'] = distance[1][0]
            wf_feature['attributes']['fire_lon'] = distance[1][1]
            wf_feature['attributes']['distance_from_del_rio'] = distance[0]
            del_rio_features.append(wf_feature)


with concurrent.futures.ThreadPoolExecutor(100) as executor:
    # Submit each fire feature for calculation
    future_to_feature = {executor.submit(calculate_closest_distance, wf_feature): wf_feature for wf_feature in gj_data['features']}

    # Wait for all calculations to complete
    for future in concurrent.futures.as_completed(future_to_feature):
        feature = future_to_feature[future]
        try:
            future.result()  # Get the result of the calculation
        except Exception as exc:
            print(f"Calculation for feature {feature} raised an exception: {exc}")

## Data Save

Saving the extracted featurs/fires in JSON format

In [8]:
import json
with open('/Users/stlp/Documents/DATA-512/del_rio_fires.json', 'w') as fout:
    json.dump(del_rio_features , fout)

Counting the number of fires within 1250 miles which started after 1963. I found there are 70861 fires based on my city Del Rio in Texas.

In [9]:
len(del_rio_features)

70861

Getting the JSON features into a dataframe using for loop to append individual features and its attributres. 

In [25]:
del_rio_fires_df = pd.DataFrame()
for i in range(len(del_rio_features)):
    if(i%10000 == 0):
        print(i)
    del_rio_fires_df = pd.concat([del_rio_fires_df, pd.DataFrame([del_rio_features[i]['attributes']])])

0
10000
20000
30000
40000
50000
60000
70000


In [193]:
del_rio_fires_df = del_rio_fires_df.reset_index(drop=True)
del_rio_fires_df.head(10)

,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,fire_lat,fire_lon,distance_from_del_rio,smoke_estimate
0,14306,14306,Wildfire,1963,1,"1 (1), 3 (1)",4995.253626,2021.507422,Comb_National_NIFC_Interagency_Fire_Perimeter_...,Wildfire (2),...,"Caution, this Wildfire in 1963 overlaps with a...",0.994707,1,Yes,15980.673439,2.021507e+07,36.475031,-106.702400,594.570587,84.014476
1,14305,14305,Wildfire,1963,1,"1 (3), 3 (3)",4995.910129,2021.773099,Comb_National_NIFC_Interagency_Fire_Perimeter_...,Wildfire (6),...,None,0.994948,1,Yes,15979.785579,2.021773e+07,35.678581,-106.479715,543.334644,91.949044
2,14341,14341,Wildfire,1963,1,"1 (2), 3 (2)",647.246513,261.931371,Comb_National_NIFC_Interagency_Fire_Perimeter_...,Wildfire (4),...,None,0.272200,None,No,10996.509615,2.619314e+06,43.526843,-103.471741,986.598492,6.560384
3,14308,14308,Wildfire,1963,1,"1 (1), 3 (2)",4293.410613,1737.481631,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (2)",...,None,0.344648,None,No,25169.664017,1.737482e+07,43.154410,-113.233561,1171.237153,36.657056
4,14324,14324,Wildfire,1963,1,"1 (1), 3 (3)",1552.699889,628.355352,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (3), Likely Wildfire (1)",...,None,0.709395,None,No,10550.269932,6.283554e+06,34.728314,-119.946244,1175.406549,13.209897
5,14343,14343,Wildfire,1963,1,"1 (1), 3 (2)",632.343510,255.900339,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (2)",...,None,0.432292,None,No,8624.859107,2.559003e+06,42.892318,-114.093802,1186.287077,5.330443
6,14312,14312,Wildfire,1963,1,"1 (1), 3 (2)",2840.558722,1149.533331,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (2)",...,None,0.535192,None,No,16428.993577,1.149533e+07,43.988438,-111.669798,1170.063244,24.276967
7,14323,14323,Wildfire,1963,1,"1 (1), 3 (2)",1562.805612,632.444993,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (2)",...,None,0.705143,None,No,10616.410202,6.324450e+06,43.261508,-113.692527,1191.857611,13.112352
8,14326,14326,Wildfire,1963,1,"1 (1), 3 (2)",1528.187835,618.435676,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (2)",...,None,0.596885,None,No,11410.557262,6.184357e+06,42.491152,-113.244755,1136.807835,13.442798
9,14369,14369,Wildfire,1963,1,"1 (2), 3 (6)",359.191740,145.359740,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (6), Likely Wildfire (2)",...,None,0.420387,None,No,6591.775927,1.453597e+06,33.884689,-116.771532,985.255672,3.645670


Saving the dataframe as CSV and Parquet file for further analysis

In [197]:
del_rio_fires_df.to_csv("del_rio_fires_df.csv")

In [202]:
del_rio_fires_df.to_parquet("del_rio_fires_df.pqt")